In [1]:
import operator
import re

from pyspark import SparkConf, SparkContext

In [2]:
sc = SparkContext(conf=SparkConf().setAppName("FindPath").setMaster("local"))

In [3]:
def parse_edge(s):
    user, follower = s.split("\t")
    return (int(user), int(follower))

def f_step(item):
    prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
    return (next_v, prev_d + 1)

def complete(item):
    v, old_d, new_d = item[0], item[1][0], item[1][1]
    return (v, old_d if old_d is not None else new_d)

n = 400  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()

In [4]:
x = 12
y = 34
d = 0

distances = sc.parallelize([(x, d)]).partitionBy(n)

while True:

    candidates = distances.filter(lambda item: item[1] == d).join(forward_edges, n).map(f_step)
    new_distances = distances.fullOuterJoin(candidates, n).map(complete, True).persist()
    
    if candidates.isEmpty():
        break

    d += 1
    distances = new_distances

    if not candidates.filter(lambda pair: pair[0] == y).isEmpty():
        break


In [5]:
path = [y]

step = d - 1
next_ = y

dist_cache = distances.cache()

while step:
    step_candidats = dist_cache.filter(lambda pair: pair[1] == step).map(lambda item: (item[0], next_))
    next_ = forward_edges.filter(lambda pair: pair[1] == next_).intersection(step_candidats).first()[0]
    # decrease step and found next
    step -= 1
    path.append(next_)
        
path.append(x)

path.reverse()
print ','.join(map(str, path))

12,422,53,52,107,20,23,274,34
